## PROJETO PÁSCOA COM DADOS
#### Apoiei o Projeto Dado Humanizado do Prof. Cláudio Bonel. Foi uma iniciativa chamada "Páscoa com Dados", que entregou ovos de páscoa para 100 crianças do Complexo do Alemão/RJ em 2022.

In [226]:
# Importando as bibliotecas necessarias
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

In [227]:
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')
dtCarga

'21/04/2022 12:04'

### COLETA DOS DADOS DOS REGISTROS DE OCORRÊNCIAS

In [228]:
# Manipulando o sistema de arquivos no Windows
LocalArquivo = 'E:\\Projetos\\Python\\08 - Prof Bonel\\Python Integracao Dados com SQL\\'

#Criar Data Frame com os dados dos registros das ocorrências
tbLogOcorrencias = pd.DataFrame(pd.read_csv(LocalArquivo + "OcorrenciaV2.csv"))

#Criando coluna com a data da carga
tbLogOcorrencias['dtCarga'] = dtCarga

tbLogOcorrencias.head()

,COD_DP,COD_BPM,ano,mes,mes_ano,Regiao,COD_Munic_IBGE,Ocorrencia,Soma de Qtde,dtCarga
0,1,5,2018,1,2018m01,1,3304557,ameaca,7,21/04/2022 12:04
1,1,5,2018,1,2018m01,1,3304557,apreensao_drogas,3,21/04/2022 12:04
2,1,5,2018,1,2018m01,1,3304557,estelionato,81,21/04/2022 12:04
3,1,5,2018,1,2018m01,1,3304557,estupro,1,21/04/2022 12:04
4,1,5,2018,1,2018m01,1,3304557,extorsao,3,21/04/2022 12:04


In [229]:
# dimensao
tbLogOcorrencias.shape

(124008, 10)

In [230]:
# Informações
tbLogOcorrencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124008 entries, 0 to 124007
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   COD_DP          124008 non-null  int64 
 1   COD_BPM         124008 non-null  int64 
 2   ano             124008 non-null  int64 
 3   mes             124008 non-null  int64 
 4   mes_ano         124008 non-null  object
 5   Regiao          124008 non-null  int64 
 6   COD_Munic_IBGE  124008 non-null  int64 
 7   Ocorrencia      124008 non-null  object
 8   Soma de Qtde    124008 non-null  int64 
 9   dtCarga         124008 non-null  object
dtypes: int64(7), object(3)
memory usage: 9.5+ MB


In [231]:
# Verificando a existencia de nulos
tbLogOcorrencias.isnull().sum()

COD_DP            0
COD_BPM           0
ano               0
mes               0
mes_ano           0
Regiao            0
COD_Munic_IBGE    0
Ocorrencia        0
Soma de Qtde      0
dtCarga           0
dtype: int64

### CONECTANDO NO ODS E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [232]:
# Manipulando o sistema de arquivos no Windows
endereco = Path('E:\\Projetos\\Python\\08 - Prof Bonel\\Python Integracao Dados com SQL\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

# Conectando no BDODS
conexaoODS = sql.connect(BDODS)

# Definição de manipulação de dados em SQL. Métdo cursor permite manipular dados no SQL
SQL_ODS = conexaoODS.cursor()

### CRIANDO A TABELA tbLogOcorrencias, no SQLite, via SQL

In [233]:
# definição da query de criação da tbLogOcorrrencias

qry_create_tbLogOcorrencias = '''

    CREATE TABLE IF NOT EXISTS tbLogOcorrencias (
    
    codDP INTEGER,
    codBPM INTEGER,
    ano INTEGER,
    mes INTEGER,
    mes_ano CHAR(7),
    regiao INTEGER,
    codMunicIBGE INTEGER,
    ocorrencias VARCHAR(100),
    qtde INTEGER,
    dtCarga DATETIME
    
    )

'''

# criando a tbLogOcorrencias
SQL_ODS.execute(qry_create_tbLogOcorrencias)

# Criação de index
SQL_ODS.execute("CREATE INDEX IF NOT EXISTS idx_tbLogOcorrencias ON tbLogOcorrencias (codDP,codBPM,codMunicIBGE)")

# confirmar a transação
conexaoODS.commit()

### CARREGANDO DADOS NA tbLogOcorrencias

In [234]:
# inserindo dados na tbLogOcorrencias
print("Iniciando a carga de dados na tbLogOcorrencias")
print("-----------------------------------------------")
SQL_ODS.executemany('''

    INSERT INTO tbLogOcorrencias (codDP,codBPM,ano,mes,mes_ano,regiao,codMunicIBGE,ocorrencias,qtde,dtCarga)
    VALUES(?,?,?,?,?,?,?,?,?,?)    
    ''', 
    tbLogOcorrencias.values.tolist()

)

# confirmar a transação
conexaoODS.commit()

print("Fim da carga de dados da tbLogOcorrencias!",len(tbLogOcorrencias),"Registros inseridos!")

Iniciando a carga de dados na tbLogOcorrencias
-----------------------------------------------
Fim da carga de dados da tbLogOcorrencias! 124008 Registros inseridos!


### CRIANDO O DATAFRAME COM O RESULTADO DA QUERY DE CARGA DE DADOS DAS OCORRÊNCIAS

In [235]:
# definição da query de carga de dados

qry_fOcorrencias = '''

    SELECT
        a.codDP,
        a.codBPM,
        a.ano,
        a.mes,
        a.regiao,
        a.codMunicIBGE,
        a.ocorrencias,
        a.qtde
    FROM tbLogOcorrencias a
    WHERE a.dtCarga = (
                        SELECT MAX(dtCarga)
                        FROM tbLogOcorrencias
                        )

'''

#popular o dataframe
fOcorrencias = pd.read_sql(qry_fOcorrencias,conexaoODS)

fOcorrencias.head()

,codDP,codBPM,ano,mes,regiao,codMunicIBGE,ocorrencias,qtde
0,1,5,2018,1,1,3304557,ameaca,7
1,1,5,2018,1,1,3304557,apreensao_drogas,3
2,1,5,2018,1,1,3304557,estelionato,81
3,1,5,2018,1,1,3304557,estupro,1
4,1,5,2018,1,1,3304557,extorsao,3


### FECHANDO A CONEXÃO COM O ODS

In [236]:
#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

### CONECTANDO NO DW E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [237]:
# Conectar no BDDW
conexaoDW = sql.connect(BDDW)

# Definição de manipulação de dados em SQL
SQL_DW = conexaoDW.cursor()

### CRIANDO A TABELA FATO OCORRENCIAS, NO DW

In [238]:
# definindo a qry da criação da fOcorrencias

qry_create_fOcorrencias = '''

    CREATE TABLE IF NOT EXISTS fOcorrencias (
    
    idDP INTEGER REFERENCES dDP(idDP) ON UPDATE NO ACTION ON DELETE NO ACTION,
    idBPM INTEGER REFERENCES dBPM(idBPM) ON UPDATE NO ACTION ON DELETE NO ACTION,
    idPeriodo INTEGER REFERENCES dPeriodo(idPeriodo) ON UPDATE NO ACTION ON DELETE NO ACTION,
    regiao INTEGER,
    idMunic INTEGER REFERENCES dMunicipio(idMuni) ON UPDATE NO ACTION ON DELETE NO ACTION,
    ocorrencias VARCHAR(100),
    qtde INTEGER
    
    )

'''

# criação da fOcorrencias
SQL_DW.execute(qry_create_fOcorrencias)

# criação de index
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_fOcorrencias ON fOcorrencias(idDP,idBPM,idPeriodo,idMunic)")
               
# confirmar a transação
conexaoDW.commit()

### INSERINDO DADOS NA FATO OCORRÊNCIAS (fOcorrencias) - CARGA INCREMENTAL

In [239]:
# criar um tabela temporária para comparar a existencia de dados novos
fOcorrencias.to_sql('tempOcorrencias',conexaoDW,if_exists="replace")

In [240]:
qry_carga_incremental_focorrencias = '''

    SELECT
        b.idDP,
        c.idBPM,
        d.idPeriodo,
        a.regiao,
        e.idMunic,
        a.ocorrencias,
        a.qtde
    FROM tempOcorrencias a
    JOIN dDP b
    ON a.codDP = b.codDP
    JOIN dBPM c
    ON a.codBPM = c.codBPM
    JOIN dPeriodo d
    ON (a.ano = d.ano) AND (a.mes = d.mes)
    JOIN dMunicipio e
    ON a.codMunicIBGE = e.codMunic
    LEFT JOIN fOcorrencias g
    ON b.idDP = g.idDP
    AND c.idBPM = g.idBPM
    AND d.idPeriodo = g.idPeriodo
    AND e.idMunic = g.idMunic
    WHERE g.idDP is null
    AND g.idBPM is null
    AND g.idPeriodo is null
    AND g.idMunic is null

'''

#criando um DF Incremental
carga_fOcorrencias = pd.read_sql(qry_carga_incremental_focorrencias,conexaoDW)

carga_fOcorrencias

,idDP,idBPM,idPeriodo,regiao,idMunic,ocorrencias,qtde


In [241]:
# carregando dados na tabela fOcorrencias
print("Carregando dados na tabela fOcorrencias.")
print("----------------------------------------")

SQL_DW.executemany('''

    INSERT INTO fOcorrencias (idDP,idBPM,idPeriodo,regiao,idMunic,ocorrencias,qtde)
    VALUES(?,?,?,?,?,?,?)''',
    
    carga_fOcorrencias.values.tolist()
)



# confirmar a transação
conexaoDW.commit()

print("Fim da carga incremental de dados na fOcorrencias!",len(carga_fOcorrencias),"registros inseridos!")

Carregando dados na tabela fOcorrencias.
----------------------------------------
Fim da carga incremental de dados na fOcorrencias! 0 registros inseridos!


### ATUALIZANDO DADOS NA FATO OCORRÊNCIAS (fOcorrencias) - ATUALIZAÇÃO RETROATIVA EM REGISTROS EXISTENTES

In [242]:
# query para verificar se tem dados atualizados na base que ja existia

qry_atualizacao_fOcorrencias = '''
    SELECT
        a.qtde,
        b.idDP,
        c.idBPM,
        d.idPeriodo,
        a.regiao,
        e.idMunic,
        a.ocorrencias        
    FROM tempOcorrencias a
    JOIN dDP b
    ON a.codDP = b.codDP
    JOIN dBPM c
    ON a.codBPM = c.codBPM
    JOIN dPeriodo d
    ON (a.ano = d.ano) AND (a.mes = d.mes)
    JOIN dMunicipio e
    ON a.codMunicIBGE = e.codMunic
    LEFT JOIN fOcorrencias g
    ON b.idDP = g.idDP
    AND c.idBPM = g.idBPM
    AND d.idPeriodo = g.idPeriodo
    AND e.idMunic = g.idMunic
    AND a.regiao = g.regiao
    AND a.ocorrencias = g.ocorrencias
    WHERE a.qtde <> g.qtde
'''

# criando um DataFrame de atualização
atualiza_fOcorrencias = pd.read_sql(qry_atualizacao_fOcorrencias, conexaoDW)

atualiza_fOcorrencias

,qtde,idDP,idBPM,idPeriodo,regiao,idMunic,ocorrencias
0,7,1,4,1,1,68,ameaca
1,3,1,4,1,1,68,apreensao_drogas


In [243]:
# Atualizando os dados retroativos na tabela fOcorrencias
print("Iniciando atualização na tabela fato Ocorrencias!")

# definindo a query de atualização
qry_atualiza_fOcorrencias = '''
    
    UPDATE fOcorrencias
    SET qtde = ?
    WHERE idDP = ?
    AND idBPM = ?
    AND idPeriodo = ?
    AND regiao = ?
    AND idMunic = ?
    AND ocorrencias = ?

'''

# atualizando dados na fOcorrencias
SQL_DW.executemany(qry_atualiza_fOcorrencias, atualiza_fOcorrencias.values.tolist())

# confirmar a transação
conexaoDW.commit()

print("Fim da atualização de dados na tabela fato Ocorrencias!", len(atualiza_fOcorrencias), "registros atualizados!")

Iniciando atualização na tabela fato Ocorrencias!
Fim da atualização de dados na tabela fato Ocorrencias! 2 registros atualizados!


In [244]:
# deletar a tabela temporária
SQL_DW.execute("DROP TABLE tempOcorrencias")

# Confirmar a transação
conexaoDW.commit()

### FECHANDO A CONEXÃO COM O DW

In [245]:
# Fechar a conexão
conexaoDW.close()

In [246]:
!ls

01 - WebScrapingDadosIBGEdoRJ.ipynb
02 - Tabela tbLogMunic criada para manter o historico.png
03 - Tabela dMunicipio criada para manter os dados atualizados.png
04 - CargaDimensaoDelegaciasPolicia.ipynb
05 - IntegrandoDadosIBGE_DP_BPM.ipynb
06 - IntegracaoDadosIBGE_DP_BPM_PERIODO.ipynb
07 - IntegracaoDadosIBGE_DP_BPM_PERIODO_OCORRENCIAS.ipynb
BPM.xlsx
DP.csv
OcorrenciaV2.csv
ResponsavelDP.csv
areaBPMv1.csv
pascoaDW.db
pascoaODS.db
popDP.csv
